In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv("Fulldocumentword2.csv" , header=None)
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,0,0,"""""","""""","""""",1470337707,"""""",1470337707_6684,"""""","""کرونا کرونا کرونا""","""کرونا کرونا کرونا""",1398-11-13 20:40:13,1580663413,"""1""",کرونا کرونا کرونا,"['کرونا', 'کرونا', 'کرونا']"
1,2,2,"""""","""یارانه""","""""",1462058741,"""""",1462058741_58843,"""ابراز علاقه""","""‌ ‌ ‌‌‌ ‌ کرونا کرونا ~~~~ چی میگی میگم دوست ...","""کرونا کرونا چی میگی میگم دوست دارم چصو کرونا ...",1398-11-18 20:39:02,1581095342,"""2""",کرونا کرونا چی میگی میگم دوست دارم چصو کرونا ک...,"['کرونا', 'کرونا', 'چی', 'میگ', 'میگ', 'دوست',..."
2,3,3,"""""","""""","""""",1497732081,"""""",1497732081_549,"""""","""😷کرونا اینجا کرونا اونجا کرونا همه جا.""","""کرونا اینجا کرونا اونجا کرونا همه جا""",1398-11-14 16:47:33,1580735853,"""57""",کرونا اینجا کرونا اونجا کرونا همه جا,"['کرونا', 'اینجا', 'کرونا', 'اونجا', 'کرونا', ..."
3,4,4,"""""","""""","""""",1022917137,"""ranginak97""",1022917137_116487,"""""","""😷کرونا اینجا کرونا اونجا کرونا همه جا. 💉آزمای...","""کرونا اینجا کرونا اونجا کرونا همه جا ازمایش و...",1398-11-12 14:49:57,1580555997,"""168""",کرونا اینجا کرونا اونجا کرونا همه جا ازمایش وا...,"['کرونا', 'اینجا', 'کرونا', 'اونجا', 'کرونا', ..."
4,5,5,"""""","""""","""""",1204349583,"""""",1204349583_6792,"""حالت تهوع""","""‏-وای چقدر سرم درد می کنه +کرونا گرفتی -حالت ...","""وای چقدر سرم درد می کنه کرونا گرفتی حالت تهوع...",1398-11-14 13:25:34,1580723734,"""8""",وای چقدر سرم درد می کنه کرونا گرفتی حالت تهوع ...,"['وای', 'چقدر', 'سرم', 'درد', 'می', 'کنه', 'کر..."


In [3]:
df[df.columns[-1]]

0                               ['کرونا', 'کرونا', 'کرونا']
1         ['کرونا', 'کرونا', 'چی', 'میگ', 'میگ', 'دوست',...
2         ['کرونا', 'اینجا', 'کرونا', 'اونجا', 'کرونا', ...
3         ['کرونا', 'اینجا', 'کرونا', 'اونجا', 'کرونا', ...
4         ['وای', 'چقدر', 'سرم', 'درد', 'می', 'کنه', 'کر...
                                ...                        
182506    ['قدردانی', 'مامور', 'پلیس', 'ووهان', 'از', 'ت...
182507    ['دو', 'سخنرانی', 'در', 'تهران', 'و', 'واشنگتن...
182508    ['اخ', 'سفید', 'قرار', 'داشت&دارد', 'که', 'نوس...
182509    ['روز', 'اول', 'سفر', 'چند', 'سال', 'بود&باش',...
182510    ['ری', 'نه', 'تنها', 'به', 'اثبات', 'رسیده', '...
Name: 15, Length: 182511, dtype: object

In [4]:
from sklearn.feature_extraction.text import CountVectorizer

In [5]:
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(df[df.columns[-1]])

In [6]:
X_train_counts.shape

(182511, 50549)

In [7]:
def spy_sparse2torch_sparse(data):
    """

    :param data: a scipy sparse csr matrix
    :return: a sparse torch tensor
    """
    samples=data.shape[0]
    features=data.shape[1]
    values=data.data
    coo_data=data.tocoo()
    indices=torch.LongTensor([coo_data.row,coo_data.col])
    t=torch.sparse.FloatTensor(indices,torch.from_numpy(values).float(),[samples,features])
    return t

## AutoEncoder

In [8]:
import torch
import torchvision
from torch import nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import MNIST
from torchvision.utils import save_image

In [9]:
# torch.cuda.is_available()
device = torch.device("cuda:0")
# device

torch.cuda.set_device(0)

# torch.cuda.current_device()
# torch.cuda.device_count()

torch.cuda.get_device_name(torch.cuda.current_device())

'GeForce GTX 1070'

In [10]:
input_dim = X_train_counts.shape[1]
encoding_dim = 32


In [11]:
input_dim

50549

In [12]:
num_epochs = 10
batch_size = 2048
learning_rate = 1e-3

In [13]:

data = spy_sparse2torch_sparse(X_train_counts)

In [14]:
dataloader = DataLoader(data, batch_size=batch_size, shuffle=True)

In [16]:
# class autoencoder(nn.Module):
#     def __init__(self):
#         super(autoencoder , self).__init__()
        
#         self.encoder = nn.Sequential(
#             nn.Linear(input_dim , 512),
#             nn.ReLU(True),
#             nn.Linear(512 , 256),
#             nn.ReLU(True),
#             nn.Linear(256 , encoding_dim))
        
#         self.decoder = nn.Sequential(
#             nn.Linear(encoding_dim , 256),
#             nn.ReLU(True),
#             nn.Linear(256 , 512),
#             nn.ReLU(True),
#             nn.Linear(512 , input_dim),
#             nn.Tanh())
        
#     def forward(self , x):
#         x = self.encoder(x)
#         x = self.decoder(x)
#         return x
    
class autoencoder(nn.Module):
    def __init__(self):
        super(autoencoder , self).__init__()
        
        self.encoder = nn.Sequential(
            nn.Linear(input_dim , 64),
            nn.ReLU(True),
            nn.Linear(64 , encoding_dim))
        
        self.decoder = nn.Sequential(
            nn.Linear(encoding_dim , 64),
            nn.ReLU(True),
            nn.Linear(64 , input_dim),
            nn.Tanh())
        
    def forward(self , x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x
    

In [17]:
import time
torch.rand(1).to(device)

tensor([0.4965], device='cuda:0')

In [ ]:
from torchsummary import summary

model = autoencoder().cuda()
#print(summary(model, (input_dim,) ))
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(
    model.parameters() , lr = learning_rate , weight_decay=1e-5)

for epoch in range(num_epochs):
    print("in")
    i=0
    for data in dataloader:
        t = time.time();
        inpx = data.to_dense()
        inpx = Variable(inpx).cuda()#using gpu vram
        
        #-----------forward-----------
        output = model(inpx)
        loss = criterion(output , inpx)
        
        #-------------backward--------
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if(i%10==0):
            print("time pased {} to process {}".format(time.time()-t , data.shape))
        i+=1
    #------------log-----------------
    print("epoch [{}/{}] , loss:{}".
          format(epoch+1 , num_epochs,loss))
        
        

in
time pased 0.15706753730773926 to process torch.Size([2048, 50549])
time pased 0.14554405212402344 to process torch.Size([2048, 50549])
time pased 0.1360321044921875 to process torch.Size([2048, 50549])
time pased 0.13303041458129883 to process torch.Size([2048, 50549])
time pased 0.13002896308898926 to process torch.Size([2048, 50549])
time pased 0.13703083992004395 to process torch.Size([2048, 50549])
time pased 0.13403010368347168 to process torch.Size([2048, 50549])
time pased 0.13202953338623047 to process torch.Size([2048, 50549])
time pased 0.13102936744689941 to process torch.Size([2048, 50549])
epoch [1/10] , loss:0.003862112294882536
in
time pased 0.1470327377319336 to process torch.Size([2048, 50549])
time pased 0.13402986526489258 to process torch.Size([2048, 50549])
time pased 0.13503050804138184 to process torch.Size([2048, 50549])
time pased 0.13302993774414062 to process torch.Size([2048, 50549])
time pased 0.13202953338623047 to process torch.Size([2048, 50549])
tim

In [23]:
X_train_counts.shape[0]/2048


89.11669921875